# Case Study: Evaluating Congestion Control for Satellite Networks

### Scenario:
Your lab is working on a project simulating congestion control protocols on satellite networks. They have run some simulations and produced data stored in Github. They have written some scripts to help with creating plots, but there are some issues they want you to take a look at because they heard you are taking ***Programming With Data*** as part of ***PCT***. Follow the tasks below and help your lab make good decisions.

You don’t need any prior knowledge about satellite networks or congestion control protocols to complete any of the examples.

#### Task
Look through `plot_multi_throughput.py` script and understand what is being done

#### Task
Use `time_scripts.py` to see the example run times when using csv vs parquet

#### Task
Decide which file format you want to use when running `generate_plots.sh`


## Task 1: Download the Data

In [ ]:
# download and extract tar file
!wget https://raw.githubusercontent.com/mitdbg/practical-programming-with-data/main/2026/day1_materials/sim_logs.tar.gz
!tar -xzf sim_logs.tar.gz

In [ ]:
# examine the extracted files
!ls sim_logs

## Task 2:
Somebody suggested to your labmate that they use the `parquet` file format to store the large logs produced by the simulations. Find out why they may have made this suggestion by looking at some differences in the performance of `parquet` vs `csv`.

In [ ]:
import pandas as pd

# read parquet file
df = pd.read_parquet("sim_logs/LeoSharedPath-1ms-TcpBbr-throughput.parquet")

# save data as csv
df.to_csv("sim_logs/LeoSharedPath-1ms-TcpBbr-throughput.csv", index=False)

In [ ]:
# first, let's look at the difference in the files' raw size(s)
!ls -lth sim_logs/ | grep LeoSharedPath-1ms-TcpBbr-throughput

In [ ]:
# next, let's time how long it takes for us to read each file from disk
import time

start_time = time.time()
df = pd.read_parquet("sim_logs/LeoSharedPath-1ms-TcpBbr-throughput.parquet")
parquet_read_time = time.time() - start_time

start_time = time.time()
df = pd.read_csv("sim_logs/LeoSharedPath-1ms-TcpBbr-throughput.csv")
csv_read_time = time.time() - start_time

print(f"parquet read time: {parquet_read_time}")
print(f"csv read time: {csv_read_time}")
print("---")
print(f"speedup: {csv_read_time / parquet_read_time:.2f}x")

In [ ]:
# now let's examine the first few rows of the dataframe
df.head()

In [ ]:
# parquet enables you to only read / load certain columns from the raw data;
# as a final experiment, let's see how long it takes us to load the flowId and bytes
# columns with parquet, relative to loading the entire CSV and post-filtering it
# next, let's time how long it takes for us to read each file from disk
import time

start_time = time.time()
df = pd.read_parquet("sim_logs/LeoSharedPath-1ms-TcpBbr-throughput.parquet", columns=["flowId", "bytes"])
parquet_read_time = time.time() - start_time

start_time = time.time()
df = pd.read_csv("sim_logs/LeoSharedPath-1ms-TcpBbr-throughput.csv")
df = df.loc[:, ["flowId", "bytes"]]
csv_read_time = time.time() - start_time

print(f"parquet read time: {parquet_read_time}")
print(f"csv read time: {csv_read_time}")
print("---")
print(f"speedup: {csv_read_time / parquet_read_time:.2f}")

## Task 3:
Let's examine the queueing delay associated with the BBR algorithm (`TcpBbr`) in the `GsToGs` setting.

In [ ]:
delay_df = pd.read_parquet("sim_logs/LeoGsToGs-1ms-TcpBbr-delay.parquet")

In [ ]:
delay_df.head()

In [ ]:
# Q: How many unique flows are in this data sample?

In [ ]:
# Data Cleaning: There's a typo in the column name "queue_dealy(us)" rename this column to "queue_delay(us)"

In [ ]:
# Plotting: plot the queueing delay in **milliseconds** vs. time in seconds

In [ ]:
# Plotting: to better examine the queueing delay behavior, generate another
# version of the plot which does not contain samples with >1.0 ms of delay

## Task 4:

Let's now examine the congestion window of various congestion control algorithms for the `GsToGs` setting.

In [ ]:
# load the congestion window (cwnd) data for `TcpBbr`, `TcpCubic`, `TcpNewReno`, and `TcpHybla` in the `GsToGs` setting
bbr_df = pd.read_parquet("sim_logs/LeoGsToGs-1ms-TcpBbr-cwnd.parquet")
cubic_df = pd.read_parquet("sim_logs/LeoGsToGs-1ms-TcpCubic-cwnd.parquet")
reno_df = pd.read_parquet("sim_logs/LeoGsToGs-1ms-TcpNewReno-cwnd.parquet")
hybla_df = pd.read_parquet("sim_logs/LeoGsToGs-1ms-TcpHybla-cwnd.parquet")

In [ ]:
# let's examine the head of each dataframe to ensure their schemas match
print(bbr_df.head())
print("---")
print(cubic_df.head())
print("---")
print(reno_df.head())
print("---")
print(hybla_df.head())

In [ ]:
# Task: coalesce all the data into a single dataframe, adding a new column 'cc_algo',
# which stores the name of the congestion control algorithm associated with that data;

In [ ]:
# Task: Remove outliers by filtering for congestion windows < 3e5

In [ ]:
# Plotting: plot the congestion window as a function of time for each algorithm;
# plot each algorithm as a separate line on the same plot w/colors and a legend to distinguish them

In [ ]:
# Task: bucket the data into 10s time intervals, write a function which computes time(s) // 10
# and apply that function to the time(s) column, saving the output in a new "interval" column

In [ ]:
# Task: compute and print the mean congestion window size for each unique (cc_algo, interval) (Hint: use .groupby())

In [ ]:
# Task: in how many intervals does TcpCubic have the largest mean congestion window?
#
# Hint:
# - Initialize two empty lists, one to contain data for TcpCubic and another to contain data for the other algorithms
# - Use the same groupby from before to compute the mean congestion window for each algorithm in each interval
# - Put each interval and congestion window into a dictionary:
#   - for cubic data: {"interval": interval, "mean_cwnd_cubic": group_df['cwnd'].mean()}
#   - for the rest: {"interval": interval, "mean_cwnd": group_df['cwnd'].mean()}
#   - append the dictionary to its respective list (cubic data --> cubic list, other data --> other list)
# - Construct a dataframe from each list
# - Join the dataframes on the "interval" column (use .merge()) and store the output in a final dataframe
# - Group the final by interval, and compute the fraction of intervals with mean_cwnd_cubic > mean_cwnd for all rows